In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 
import shutil

def rmtree(f: Path):
    if f.is_file():
        f.unlink()
    else:
        for child in f.iterdir():
            rmtree(child)
        f.rmdir()

In [3]:
base_path = Path('data/raw_data')
files = list(base_path.glob("*/*/*.*"))
files

[PosixPath('data/raw_data/tanpa_bola/take_269663/right_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_269663/left_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_441446/right_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_441446/New Text Document.txt'),
 PosixPath('data/raw_data/tanpa_bola/take_441446/left_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_238608/right_cam.jpg'),
 PosixPath('data/raw_data/tanpa_bola/take_238608/left_cam.jpg'),
 PosixPath('data/raw_data/bola/take_700692/right_cam.jpg'),
 PosixPath('data/raw_data/bola/take_700692/New Text Document.txt'),
 PosixPath('data/raw_data/bola/take_700692/left_cam.jpg'),
 PosixPath('data/raw_data/bola/take_086448/right_cam.jpg'),
 PosixPath('data/raw_data/bola/take_086448/New Text Document.txt'),
 PosixPath('data/raw_data/bola/take_086448/left_cam.jpg'),
 PosixPath('data/raw_data/bola/take_152294/right_cam.jpg'),
 PosixPath('data/raw_data/bola/take_152294/New Text Document.txt'),
 PosixPath('data/raw_data/bola/

In [4]:

# clean data
base_clean = base_path.parent.joinpath('cleaned')
base_clean.mkdir(parents=True, exist_ok=True)
rmtree(base_clean)


# run copy
group = []
for fpath in files:
    clazz = fpath.parent.parent.stem
    parent = fpath.parent.stem
    dst_base = base_clean.joinpath(clazz).joinpath(parent)
    dst_base.mkdir(parents=True, exist_ok=True)
    if fpath.suffix=='.txt':
        # copy note to destination directory
        dst = dst_base.joinpath('note.txt')
        shutil.copy(fpath, dst)
        
         # add [DEFAULT] to the top of the file
        with open(dst, 'r') as original: data = original.read()
        with open(dst, 'w') as modified: modified.write("[default]\n" + data)
    else:
        dst = dst_base.joinpath(fpath.name)
        shutil.copy(fpath, dst)
        
    

In [1]:
# !pip install imutils

Processing /home/nunenuh/.cache/pip/wheels/59/1b/52/0dea905f8278d5514dc4d0be5e251967f8681670cadd3dca89/imutils-0.5.4-py3-none-any.whl
You should consider upgrading via the '/workspace/study/code/repo/hildanida/skripsi/venv/bin/python -m pip install --upgrade pip' command.


In [221]:
import configparser
import cv2 as cv
import imutils
    
def get_note(fpath: Path):
    config = configparser.ConfigParser(default_section='note')
    config.read(str(fpath))
    return dict(config['default'])

def preprocess_image(left_path, right_path):
    left_img = cv.imread(left_path)
    left_img = cv.cvtColor(left_img, cv.COLOR_BGR2GRAY)
    left_img = cv.resize(left_img, (224,224), interpolation=cv.INTER_AREA)
    left_img = left_img/255.0
    
    right_img = cv.imread(right_path)
    right_img = cv.cvtColor(right_img, cv.COLOR_BGR2GRAY)
    right_img = cv.resize(right_img, (224,224), interpolation=cv.INTER_AREA)
    right_img = right_img/255.0
    
    img = cv.merge((left_img, right_img))
    
    return img

    
    

In [6]:
# !pip install scikit-learn

In [222]:
# base data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale

def get_cleaned_data(root):
    base_path_clean = Path(root)

    files = list(base_path_clean.glob("*/*/*.*"))
    list_dir = [fpath.parent for fpath in files]
    
    images, labels = [],[]
    for pdir in list_dir:
        lcam_path = pdir.joinpath('left_cam.jpg')
        rcam_path = pdir.joinpath('right_cam.jpg')
        note_path = pdir.joinpath('note.txt')
        if note_path.exists():
            img = preprocess_image(str(lcam_path), str(rcam_path))
            dist = float(get_note(note_path).get('z',0))
        else:
            img = preprocess_image(str(lcam_path), str(rcam_path))
            dist = float(0.0)
        images.append(img)
        labels.append(dist)
    # norm_images = np.array(images)/255
    images = np.array(images)
    norm_labels = minmax_scale(np.array(labels), feature_range=(0,1)) 
    
    return images, norm_labels
    
X,y = get_cleaned_data('data/cleaned')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [225]:
import tensorflow as tf

X_train[0].shape, y_train.shape, X_test[0].shape, y_test.shape

# len(X_train), len(X_test)
# l,r = tf.split(X_train[0].reshape(1,224,224,6), num_or_size_splits=2, axis=3)
# l.shape, r.shape

((224, 224, 2), (324,), (224, 224, 2), (37,))

In [238]:
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import tensorflow as tf

class StereoDistance(Model):
	def __init__(self, classes=1):
		super(StereoDistance, self).__init__()
		self.chan_dim=-1
  
		# self.left_layer = Sequential([
		# 	layers.Conv2D(64, (7, 7), padding="same", strides=(2,2)),
		# 	layers.Activation("relu"),
		# 	layers.BatchNormalization(),
		# 	layers.MaxPooling2D(pool_size=(2, 2))
		# ])
  
		# self.right_layer = Sequential([
		# 	layers.Conv2D(64, (7, 7), padding="same", strides=(2,2)),
		# 	layers.Activation("relu"),
		# 	layers.BatchNormalization(),
		# 	layers.MaxPooling2D(pool_size=(2, 2))
		# ])

		# self.concat = layers.Concatenate(axis=3)
  

		self.layer1 = Sequential([
			layers.Conv2D(64, (7, 7), padding="same", strides=(2,2)),
			layers.Activation("relu"),
			# layers.BatchNormalization(),
			layers.MaxPooling2D(pool_size=(2, 2))
		])

		self.layer2 = Sequential([
			layers.Conv2D(64, (3, 3), padding="same"),
			layers.Activation("relu"),
			# layers.BatchNormalization(),
			layers.MaxPooling2D(pool_size=(2, 2))
		])

		self.layer3 = Sequential([
			layers.Conv2D(64, (3, 3), padding="same"),
			layers.Activation("relu"),
			# layers.BatchNormalization(),
			layers.MaxPooling2D(pool_size=(2, 2))
		])
  
		self.layer4 = Sequential([
			layers.Conv2D(64, (3, 3), padding="same"),
			layers.Activation("relu"),
			# layers.BatchNormalization(),
			layers.MaxPooling2D(pool_size=(2, 2))
		])
  
		self.avg_pool = layers.GlobalAvgPool2D()

		# self.flatten = layers.Flatten()

		self.fc1 = Sequential([
			layers.Dense(64),
			layers.Activation("relu"),
			layers.BatchNormalization(),
			layers.Dropout(0.5)
		])
  
		self.fc2 = Sequential([
			layers.Dense(classes),
			layers.Activation("relu")
		])
  
  
	def call(self, inputs):
		# left_inputs, right_inputs = tf.split(inputs, num_or_size_splits=2, axis=3)
		# left = self.left_layer(left_inputs)
		# right = self.right_layer(right_inputs)
		# x = self.concat([left, right])
		x = inputs
		x = self.layer1(x)
		x = self.layer2(x)
		x = self.layer3(x)
		x = self.layer4(x)
		x = self.avg_pool(x)
		# x = self.flatten(x)
		x = self.fc1(x)
		x = self.fc2(x)
  
		return x



In [239]:
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf

model = StereoDistance()
model.build(input_shape=(None, 224,224,2))

model.compile(optimizer=Adam(learning_rate=0.0005), 
              loss=tf.keras.losses.Huber(),
              metrics=['mse','accuracy'])

model.summary()


Model: "stereo_distance_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_473 (Sequential)  (None, 56, 56, 64)       6336      
                                                                 
 sequential_474 (Sequential)  (None, 28, 28, 64)       36928     
                                                                 
 sequential_475 (Sequential)  (None, 14, 14, 64)       36928     
                                                                 
 sequential_476 (Sequential)  (None, 7, 7, 64)         36928     
                                                                 
 sequential_477 (Sequential)  (None, 3, 3, 64)         36928     
                                                                 
 global_average_pooling2d (G  multiple                 0         
 lobalAveragePooling2D)                                          
                                                

In [169]:
# model(X_train[0])

In [240]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

history = model.fit(X_train, 
                   y_train, 
                   epochs=100, 
                   batch_size=64,
                   validation_data=(X_test, y_test))

No GPU found
Epoch 1/100


2022-11-16 07:56:42.503341: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 07:56:42.503619: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


6/6 [==============================] - 4s 472ms/step - loss: 0.3305 - mse: 0.3305 - accuracy: 0.0185 - val_loss: 0.4460 - val_mse: 0.4460 - val_accuracy: 0.0000e+00
Epoch 2/100
6/6 [==============================] - 3s 457ms/step - loss: 0.2417 - mse: 0.2417 - accuracy: 0.0340 - val_loss: 0.4460 - val_mse: 0.4460 - val_accuracy: 0.0000e+00
Epoch 3/100
6/6 [==============================] - 3s 438ms/step - loss: 0.2211 - mse: 0.2211 - accuracy: 0.0247 - val_loss: 0.4460 - val_mse: 0.4460 - val_accuracy: 0.0000e+00
Epoch 4/100
6/6 [==============================] - 3s 443ms/step - loss: 0.2040 - mse: 0.2040 - accuracy: 0.0340 - val_loss: 0.3717 - val_mse: 0.3717 - val_accuracy: 0.0000e+00
Epoch 5/100
6/6 [==============================] - 3s 465ms/step - loss: 0.2220 - mse: 0.2220 - accuracy: 0.0216 - val_loss: 0.3350 - val_mse: 0.3350 - val_accuracy: 0.0000e+00
Epoch 6/100
6/6 [==============================] - 3s 453ms/step - loss: 0.1964 - mse: 0.1964 - accuracy: 0.0278 - val_loss: 0.

KeyboardInterrupt: 